# Loading Essential Libraries and Read in Data

In [22]:
import numpy as np
import pandas as pd
import gc
# import lightgbm as lgb
# from sklearn.model_selection import StratifiedKFold
# from scipy.stats import rankdata
from sklearn.metrics import roc_auc_score

In [23]:
train_df = pd.read_csv("../input/train.csv")
test_df  = pd.read_csv("../input/test.csv")

In [24]:
# Features
features = [x for x in train_df.columns if x.startswith("var")]

In [25]:
%time
for df in [test_df, train_df]:
    df['sum'] = df[features].sum(axis=1)  
    df['min'] = df[features].min(axis=1)
    df['max'] = df[features].max(axis=1)
    df['mean'] = df[features].mean(axis=1)
    df['std'] = df[features].std(axis=1)
    df['skew'] = df[features].skew(axis=1)
    df['kurt'] = df[features].kurtosis(axis=1)
    df['med'] = df[features].median(axis=1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.54 µs


In [26]:
# # To plot distribution of newly created features

# def plot_new_feature_distribution(df1, df2, label1, label2, features):
#     i = 0
#     sns.set_style('whitegrid')
#     plt.figure()
#     fig, ax = plt.subplots(2,4,figsize=(18,8))

#     for feature in features:
#         i += 1
#         plt.subplot(2,4,i)
#         sns.kdeplot(df1[feature], bw=0.5,label=label1)
#         sns.kdeplot(df2[feature], bw=0.5,label=label2)
#         plt.xlabel(feature, fontsize=11)
#         locs, labels = plt.xticks()
#         plt.tick_params(axis='x', which='major', labelsize=8)
#         plt.tick_params(axis='y', which='major', labelsize=8)
#     plt.show();

In [27]:
# # Distribution of new features, grouped by value of corresponding target values
# t0 = train_df.loc[train_df['target'] == 0]
# t1 = train_df.loc[train_df['target'] == 1]
# features = train_df.columns.values[202:]
# plot_new_feature_distribution(t0, t1, 'target: 0', 'target: 1', features)

In [28]:
# # Distribution of new features values for train and test
# features = train_df.columns.values[202:]
# plot_new_feature_distribution(train_df, test_df, 'train', 'test', features)

In [29]:
# Add Rounded Value Features

%time
for feature in features:
    train_df['r2_'+feature] = np.round(train_df[feature], 2)
    test_df['r2_'+feature] = np.round(test_df[feature], 2)
    train_df['r1_'+feature] = np.round(train_df[feature], 1)
    test_df['r1_'+feature] = np.round(test_df[feature], 1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 20 µs


In [30]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# encoder = LabelEncoder()

# Bin Feature

for feature in features:
    train_df[feature + "_qbinned"] =\
    pd.qcut(train_df.loc[:, feature].values,
        q=10,labels=False)

In [32]:
for feature in features:
    test_df[feature + "_qbinned"] =\
    pd.qcut(test_df.loc[:, feature].values,
        q=10,labels=False)

In [33]:
# Z-score standardization

for var in features:
    train_df['zscore_' + var] =\
    (train_df[var]-train_df[var].mean())/train_df[var].std()
    
    test_df['zscore_' + var] =\
    (test_df[var]-test_df[var].mean())/test_df[var].std()

In [34]:
train_df.shape, test_df.shape

((200000, 1010), (200000, 1009))

In [ ]:
train_df.to_csv("fe_train.csv")
test_df.to_csv("fe_test.csv")